### SQLAlchemy

SQLAlchemy
- http://docs.sqlalchemy.org/en/latest/orm/tutorial.html
- `pip3 install sqlalchemy`
- python에서 사용하는 ORM

ORM이란?
- Object-relational mapping
- 데이터 베이스를 객체화 시켜 데이터베이스에 있는 데이터를 CRUD를 할수 있음.
- 사용하는 DB를 변경해야 할 경우 데이터 베이스 엔진만 바꾸면 됌

1. db connect
2. mapping class (db.table - class)
3. insert
4. select
5. update
6. delete

In [78]:
import sqlalchemy, pickle
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DateTime
from sqlalchemy.orm import sessionmaker

##### db connect

In [79]:
# mysql+mysqldb://<사용자계정>:<비밀번호>@<호스트>/<데이터베이스_이름>
pw = 'dhrnclfvkf@@12'
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@localhost/world")

##### mapping class

In [80]:
# __tablename__ : 데이터베이스의 테이블 이름

Base = declarative_base()
class User(Base):
    __tablename__ = 'user2'

    user_id = Column(Integer, primary_key=True)
    name = Column(String(20))
    email = Column(String(30))
    age = Column(Integer)
    rdate = Column(DateTime)
    
    def __init__(self, name, email, age, rdate):
        self.name = name
        self.email = email
        self.age = age
        self.rdate = rdate
    
    def __repr__(self):
        return "<User {}, {}, {}, {}>".format(self.name, self.email, self.age, self.rdate)

In [81]:
# engine에 연결된 데이터 베이스 테이블 생성
Base.metadata.create_all(engine)

C:\Users\user\Anaconda3\lib\site-packages\sqlalchemy\engine\default.py:507: Warning: (1366, "Incorrect string value: '\\xB4\\xEB\\xC7\\xD1\\xB9\\xCE...' for column 'VARIABLE_VALUE' at row 481")
  cursor.execute(statement, parameters)


In [82]:
# make session
Session = sessionmaker(bind=engine)
session = Session()

##### insert

In [83]:
# add user object to session
# add one object
user = User('jin', 'jin@gmail.com', '27', '2016-03-21')
session.add(user)

In [84]:
# add multy objects
users = [
    User('alice', 'alice@gmail.com', '25', '2018-02-21'),
    User('andy', 'andy@daum.net', '33', '2015-01-19'),
]
session.add_all(users)

In [85]:
# run transaction
session.commit()

In [86]:
# rollback (session data clear)
session.rollback()

##### select
- filter
- order by

In [92]:
def disp(datas):
    for data in datas:
        print(data)

In [93]:
results = session.query(User).all()
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [94]:
# filter
# ==, !=, >, <, >=, <=, like, in_, ~, 
results = session.query(User).filter(User.name == "jin")
disp(results)

<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>


In [95]:
results = session.query(User).filter(User.name != "jin")
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [96]:
results = session.query(User).filter(User.age > 26)
disp(results)

<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [97]:
results = session.query(User).filter(User.email.like("%gmail%"))
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>


In [98]:
results = session.query(User).filter(User.name.in_(["alice","andy"]))
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [99]:
results = session.query(User).filter(~User.email.like("%gmail%"))
disp(results)

<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [100]:
# and_, or_
from sqlalchemy import and_, or_
results = session.query(User).filter(or_(User.name == "jin", User.age == 33))
disp(results)

<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [101]:
results = session.query(User).filter(and_(User.age < 30, User.email.like("%gmail%")))
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>


In [102]:
# order_by
results = session.query(User).order_by(User.age.asc())
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [103]:
results = session.query(User).order_by(User.age)
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [104]:
results = session.query(User).order_by(User.age.desc())
disp(results)

<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>


In [105]:
# count
session.query(User).count()

7

##### update

In [111]:
jin = session.query(User).filter(User.name == "jin").one()

In [112]:
jin.age = 25

In [113]:
session.add(jin)

In [114]:
session.commit()

##### delete

In [115]:
session.query(User).filter(User.name == "jin").delete()

1

In [116]:
session.commit()

In [117]:
# close session
session.close()